In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer

# Load and Preprocess Data
filename = "mental_H.txt"
with open(filename, "r", encoding="utf-8", errors="ignore") as file:
    raw_text = file.read().lower()
import re

# Remove unnecessary characters and normalize text
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip().lower()  # Convert to lowercase
    return text

raw_text = clean_text(raw_text)

# Tokenization
##
tokenizer = Tokenizer(num_words=5000)  # Limit vocab size to 5000
tokenizer.fit_on_texts([raw_text])
##
sequences = tokenizer.texts_to_sequences([raw_text])[0]

vocab_size = len(tokenizer.word_index) + 1

##
seq_length = 150

# Create Input-Output Pairs
X = []
y = []
for i in range(0, len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

# Reshape for LSTM Input
X = np.reshape(X, (X.shape[0], X.shape[1]))

# Model Definition
##
model = Sequential([
    Embedding(vocab_size, 256, input_length=seq_length),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.3),
    BatchNormalization(),
    Bidirectional(LSTM(256)),
    Dropout(0.3),
    Dense(vocab_size, activation="softmax")
])


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 256)          1294592   
                                                                 
 bidirectional_2 (Bidirectio  (None, 150, 512)         1050624   
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 150, 512)          0         
                                                                 
 batch_normalization_1 (Batc  (None, 150, 512)         2048      
 hNormalization)                                                 
                                                                 
 bidirectional_3 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                      

In [14]:
checkpoint = ModelCheckpoint("weights-best1.hdf5", monitor="loss", save_best_only=True, verbose=1)
callbacks = [checkpoint]

# Train the Model
model.fit(X, y, epochs=100, batch_size=64, callbacks=callbacks)

Epoch 1/100
675/675 [==============================] - ETA: 0s - loss: 6.6721 - accuracy: 0.0630
Epoch 1: loss improved from inf to 6.67208, saving model to weights-best1.hdf5
675/675 [==============================] - 58s 79ms/step - loss: 6.6721 - accuracy: 0.0630
Epoch 2/100
675/675 [==============================] - ETA: 0s - loss: 6.2044 - accuracy: 0.1013
Epoch 2: loss improved from 6.67208 to 6.20436, saving model to weights-best1.hdf5
675/675 [==============================] - 54s 80ms/step - loss: 6.2044 - accuracy: 0.1013
Epoch 3/100
675/675 [==============================] - ETA: 0s - loss: 5.9117 - accuracy: 0.1215
Epoch 3: loss improved from 6.20436 to 5.91173, saving model to weights-best1.hdf5
675/675 [==============================] - 53s 79ms/step - loss: 5.9117 - accuracy: 0.1215
Epoch 4/100
674/675 [============================>.] - ETA: 0s - loss: 5.6580 - accuracy: 0.1382
Epoch 4: loss improved from 5.91173 to 5.65807, saving model to weights-best1.hdf5
675/675 [==

In [8]:
model.load_weights("weights-best1.hdf5")

# Generate Text
seed_idx = np.random.randint(0, len(X) - 1)
seed_sequence = X[seed_idx]

output = []
p=0
for _ in range(1000):  # Generate 1000 characters
    if i==".":
        p+=1
        if p==2:
            break 
    pred_input = np.reshape(seed_sequence, (1, len(seed_sequence)))
    pred_probs = model.predict(pred_input, verbose=0)
    next_idx = np.argmax(pred_probs)
    output.append(tokenizer.index_word[next_idx])

    # Update seed sequence
    seed_sequence = np.append(seed_sequence[1:], next_idx)

print("Generated Text:")
print(" ".join(output))

Generated Text:
coping problemsolving and interpersonal skills and learning to manage emotions protective and supportive environments in the family at school and in the wider community are important multiple factors affect mental health the more risk factors adolescents are exposed to the greater the potential impact on their mental health factors that can contribute to stress during adolescence include exposure to adversity pressure to conform with peers and exploration of identity media influence and gender norms can exacerbate the disparity between an adolescents lived reality and their perceptions or aspirations for the future other important determinants include the quality of their home life and relationships with peers violence especially sexual violence and bullying harsh parenting and severe and socioeconomic problems are recognized risks to mental health some adolescents are at greater risk of mental health conditions due to their living conditions stigma discrimination or ex

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained model
model.load_weights("weights-best.hdf5")

# Define the knowledge base
with open('mental_h.txt', 'r', encoding='utf-8') as file:
    knowledge_base = file.read()

# Extract knowledge base sections
def find_relevant_info(user_input, knowledge_text):
    vectorizer = TfidfVectorizer().fit_transform([knowledge_text, user_input])
    similarity = vectorizer.toarray().dot(vectorizer.toarray().T)[0, 1]
    if similarity > 0.1:
        # Extract sentences with relevance
        return '\n'.join([sentence for sentence in knowledge_text.splitlines() if user_input.lower() in sentence.lower()])
    return "I don't have specific information about that. Let's explore general advice."

# Generate a creative response
def generate_response(user_input, tokenizer, model, max_sequence_length, output_length=100):
    input_sequence = tokenizer.texts_to_sequences([user_input])
    input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='pre')

    output = []
    for _ in range(output_length):
        pred_probs = model.predict(input_sequence, verbose=0)
        next_idx = np.argmax(pred_probs)
        output.append(tokenizer.index_word.get(next_idx, ""))

        input_sequence = np.append(input_sequence[0][1:], next_idx).reshape(1, max_sequence_length)

    return " ".join(output)

# Example usage
user_input = input("You: ")
relevant_info = find_relevant_info(user_input, knowledge_base)
creative_response = generate_response(user_input, tokenizer, model, max_sequence_length=100, output_length=100)

response = f"Here is some information related to your query:\n{relevant_info}\n\nChatbot: {creative_response}"
print(response)


You:  hi


Here is some information related to your query:
I don't have specific information about that. Let's explore general advice.

Chatbot: and worry in the same risk of mental health conditions are not not have a mental health condition is not not have a mental health condition is not not have a mental illness is not not not have a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person has a person


In [7]:
model.load_weights("weights-best.hdf5")

# Generate Text
seed_idx = np.random.randint(0, len(X) - 1)
seed_sequence = X[seed_idx]

output = []
temperature = 1.0  # Lower values make text more deterministic; higher values make it more diverse.

for i in range(2000):  # Generate 1000 characters
    if i<1500:
        pred_input = np.reshape(seed_sequence, (1, len(seed_sequence)))
        pred_probs = model.predict(pred_input, verbose=0)
        next_idx = np.argmax(pred_probs)
        output.append(tokenizer.index_word[next_idx])

    # Update seed sequence
        seed_sequence = np.append(seed_sequence[1:], next_idx)
        
    else:
        if seed_idx == ".":
            break
        else:
            pred_input = np.reshape(seed_sequence, (1, len(seed_sequence)))
            pred_probs = model.predict(pred_input, verbose=0)
            next_idx = np.argmax(pred_probs)
            output.append(tokenizer.index_word[next_idx])

            # Update seed sequence
            seed_sequence = np.append(seed_sequence[1:], next_idx)
            
        

print("Generated Text:")
print("".join(output)+".")

KeyboardInterrupt: 